In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer
from pyspark.sql.functions import length
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
##dataframe = sqlContext.read.format('csv').options(header='true', inferScheme='true').load('C:/Users/jazmi/OneDrive/Documents/DataMate/db/all.csv')
df = spark.table("all_csv")

df.columns

Out[ 8 ]: ['_c0', 'username', 'Gender', 'Education', 'Ethnicity', 'about_me_text']

In [3]:
df = df.select('username', 'about_me_text', 'Gender').na.drop()
df_length = df.withColumn('length', length(df['about_me_text']))
df_length.show()

+--------------+--------------------+------+------+
 username| about_me_text|Gender|length|
+--------------+--------------------+------+------+
ShortyFire1984|....................| F| 30|
 ROXYCHIC549|'" I am not an up...| F| 1700|
 tequilagirl69|Laid back lovable...| F| 181|
atxredneckgirl|Tx country music ...| F| 284|
 mel12th|I am here to look...| F| 364|
 Andi010274|Can't get into my...| F| 49|
 nodutch|Music business go...| F| 67|
 lauranmr|Of all the dating...| F| 735|
 smbd75|Likes: fun, adven...| F| 73|
 candipie1989|Single black fema...| F| 385|
 tif1985|I'm on here just ...| F| 309|
 De1964|Please local only...| F| 1118|
 Trini1971|I am a Mom to a 1...| F| 589|
 FiaFia1213|I like to think I...| F| 352|
 SweetJojos| Up for anything| F| 15|
 Tawdra83|I’m a artist. I c...| F| 434|
 LadyTexas254|not looking for a...| F| 241|
 Vigilia79|Hi there! I'm Chr...| F| 650|
 ayoung292|Music is life and...| F| 105|
 Foreverfungrl| Ask me Are| F| 10|
+--------------+--------------------+------+------+
only showing top 20 rows

In [4]:
male_female_to_num = StringIndexer(inputCol='Gender', outputCol='label')

In [5]:
# Tokenize data
tokened_df = Tokenizer(inputCol="about_me_text", outputCol="token_text")
tt_df = tokened_df.transform(df)
tt_df.show()


+--------------+--------------------+------+--------------------+
 username| about_me_text|Gender| token_text|
+--------------+--------------------+------+--------------------+
ShortyFire1984|....................| F|[...................|
 ROXYCHIC549|'" I am not an up...| F|['", i, am, not, ...|
 tequilagirl69|Laid back lovable...| F|[laid, back, lova...|
atxredneckgirl|Tx country music ...| F|[tx, country, mus...|
 mel12th|I am here to look...| F|[i, am, here, to,...|
 Andi010274|Can't get into my...| F|[can't, get, into...|
 nodutch|Music business go...| F|[music, business,...|
 lauranmr|Of all the dating...| F|[of, all, the, da...|
 smbd75|Likes: fun, adven...| F|[likes:, fun,, ad...|
 candipie1989|Single black fema...| F|[single, black, f...|
 tif1985|I'm on here just ...| F|[i'm, on, here, j...|
 De1964|Please local only...| F|[please, local, o...|
 Trini1971|I am a Mom to a 1...| F|[i, am, a, mom, t...|
 FiaFia1213|I like to think I...| F|[i, like, to, thi...|
 SweetJojos| Up for anything| F| [up, for, anything]|
 Tawdra83|I’m a artist. I c...| F|[i’m, a, artist.,...|
 LadyTexas254|not looking for a...| F|[not, looking, fo...|
 Vigilia79|Hi there! I'm Chr...| F|[hi, there!, i'm,...|
 ayoung292|Music is life and...| F|[music, is, life,...|
 Foreverfungrl| Ask me Are| F| [ask, me, are]|
+--------------+--------------------+------+--------------------+
only showing top 20 rows

In [6]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')

In [7]:
# Run the hashing term frequency
hashing = HashingTF(inputCol='token_text', outputCol="hashedValues")

# Transform into a DF
#hashed_df = hashing.transform(tt_df)

In [8]:
#hashed_df.show()

In [9]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="idf_features")
#idfModel = idf.fit(hashed_df)
#rescaledData = idfModel.transform(hashed_df)

In [10]:
# Display the DataFrame
#rescaledData.select("words", "features").show()

In [11]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_features', 'length'], outputCol='features')

In [12]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[male_female_to_num, tokened_df, stop_remove, hashing, idf, clean_up])

In [13]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df_length)
cleaned = cleaner.transform(df_length)

In [14]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(262145,[215289,2...|
 0.0|(262145,[961,4054...|
 0.0|(262145,[4333,844...|
 0.0|(262145,[9639,154...|
 0.0|(262145,[14,7401,...|
 0.0|(262145,[6693,189...|
 0.0|(262145,[9639,572...|
 0.0|(262145,[1998,761...|
 0.0|(262145,[70082,10...|
 0.0|(262145,[8449,134...|
 0.0|(262145,[4200,576...|
 0.0|(262145,[1536,332...|
 0.0|(262145,[2711,869...|
 0.0|(262145,[329,9639...|
 0.0|(262145,[16332,10...|
 0.0|(262145,[2437,963...|
 0.0|(262145,[13396,16...|
 0.0|(262145,[456,4054...|
 0.0|(262145,[2153,262...|
 0.0|(262145,[109810,1...|
+-----+--------------------+
only showing top 20 rows

In [15]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [16]:
test_results = predictor.transform(testing)
test_results.show(5)

+--------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 username| about_me_text|Gender|length|label| token_text| stop_tokens| hashedValues| idf_features| features| rawPrediction| probability|prediction|
+--------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 1KindOfLove|Hi, my name is Sh...| F| 892| 0.0|[hi,, my, name, i...|[hi,, name, sheil...|(262144,[3358,751...|(262144,[3358,751...|(262145,[3358,751...|[-5150.0449938963...|[1.0,3.6247715258...| 0.0|
 1STREAD|I LIKE MEN....5"1...| F| 91| 0.0|[i, like, men.......|[like, men....5"1...|(262144,[24417,91...|(262144,[24417,91...|(262145,[24417,91...|[-542.97872533631...|[0.99999522284399...| 0.0|
1ofakind112633|I am an accomplis...| M| 410| 1.0|[i, am, an, accom...|[accomplished, we...|(262144,[7388,963...|(262144,[7388,963...|(262145,[7388,963...|[-2206.0284637510...|[1.0,3.5635543934...| 0.0|
 22aries74|I'm mostly a spur...| F| 425| 0.0|[i'm, mostly, a, ...|[mostly, spur, mo...|(262144,[2618,844...|(262144,[2618,844...|(262145,[2618,844...|[-2348.0569323638...|[0.99999999999999...| 0.0|
 5122027477|Spend most of my ...| M| 221| 1.0|[spend, most, of,...|[spend, time, tak...|(262144,[9639,134...|(262144,[9639,134...|(262145,[9639,134...|[-1435.6472126327...|[4.87341763978809...| 1.0|
+--------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.474217